In [1]:
import openpathsampling as paths

# Simplest Possible OPS Example

## Set up the system

### Prepare MD engine

In [2]:
# always the most complicated bit
import openpathsampling.engines.toy as toys
import numpy as np

pes = (toys.OuterWalls([1.0,1.0], [0.0,0.0]) + 
       toys.Gaussian(-0.7, [12.0, 0.5], [-0.5, 0.0]) +
       toys.Gaussian(-0.7, [12.0, 0.5], [0.5, 0.0]))
topology = toys.Topology(n_spatial=2, masses=[1.0, 1.0], pes=pes)
template = toys.Snapshot(coordinates=np.array([[0.0, 0.0]]),
                         velocities=np.array([[0.0, 0.0]]),
                         topology=topology)
engine = toys.Engine(options={'integ': toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5),
                              'n_frames_max': 5000,
                              'nsteps_per_frame': 10},
                     template=template)

### Define states

In [3]:
# states are volumes in a CV space: define the CV
def xval(snapshot):
    return snapshot.xyz[0][0]

cv = paths.CV_Function("xval", xval)

In [4]:
stateA = paths.CVRangeVolume(cv, float("-inf"), -0.5).named("A")
stateB = paths.CVRangeVolume(cv, 0.5, float("inf")).named("B")

## Prepare path sampling

### Set up the `network` and `move_scheme`

In [5]:
network = paths.TPSNetwork(stateA, stateB)

In [6]:
scheme = paths.OneWayShootingMoveScheme(network, engine=engine)

### Prepare initial conditions

In [7]:
# I'll fake an initial trajectory
trajectory = paths.Trajectory([
    toys.Snapshot(coordinates=np.array([[-.55+k*0.1, 0.0]]),
                  velocities=np.array([[0.1, 0.0]]),
                  topology=topology)
    for k in range(12)
])

In [8]:
initial_conditions = scheme.initial_conditions_from_trajectories(trajectory)

## Run path sampling

### Create storage and simulation object

In [9]:
storage = paths.Storage("simple.nc", "w", template=template)
simulation = paths.PathSampling(storage, scheme, initial_conditions)

### Run

In [10]:
simulation.run(1000)

Working on Monte Carlo cycle number 1000.

DONE! Completed 1000 Monte Carlo cycles.



## Analyze results

In [11]:
# 1. path tree

In [12]:
# 2. committor based on shooting points

In [13]:
# 3. scheme.move_summary(storage)

In [14]:
# 4. path length histogram?